<a href="https://colab.research.google.com/github/DhimanTarafdar/medical-insurance-cost-prediction-explore/blob/main/Medical_Insurance_Cost_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Regression models
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
# Metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Cross-validation and tuning
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Load the dataset
df = pd.read_csv("/content/medical_insurance.csv")

# Basic information
print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print("\n")
print("Column names:")
print(df.columns.tolist())

In [ ]:
!pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport( df , title="Medical Insurance Cost Prediction", explorative = True  )

profile.to_file("ydata.html")

In [ ]:
# Display first few rows
print("First 5 rows:")
print(df.head())

print("\n")

# Data types and missing values
print("Data types and missing values:")
print(df.info())

print("\n")

# Statistical summary
print("Statistical Summary:")
print(df.describe())

print("\n")

# Missing values count
print("Missing values count:")
print(df.isnull().sum())

print("\n")

# Check for duplicate rows
print(f"Duplicate rows: {df.duplicated().sum()}")

In [ ]:
df[df.duplicated()]


In [ ]:
df = df.drop_duplicates()

In [ ]:
print("Duplicate rows after removal:", df.duplicated().sum())

In [ ]:
df.shape

In [ ]:
# Check unique values in categorical columns

print("Unique values in categorical columns:")

print(f"Sex: {df['sex'].nunique()} unique sex")
print(f"Region: {df['region'].nunique()} unique region")
print(f"Smoker: {df['smoker'].nunique()} unique smoker")


# Check some categorical column values
print("Sex Values:")
print(df['sex'].value_counts())

print("\nRegion Values:")
print(df['region'].value_counts())

print("\nSmoker Values:")
print(df['smoker'].value_counts())



In [ ]:
# Target variable distribution
print("Target variable (charges) distribution:")
print(df['charges'].describe())

print("\n")

# Correlation with target (numeric features only)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
corr_with_charges= df[numeric_cols].corr()['charges'].sort_values(ascending=False)

print("Correlation with charges:")
print(corr_with_charges)

print("\n")

# Visualize charges distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(df['charges'], bins=50, color='skyblue', edgecolor='black')
plt.xlabel('charges')
plt.ylabel('Frequency')
plt.title('charges Distribution')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(df['charges'])
plt.ylabel('charges')
plt.title('charges Boxplot (Outlier Check)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n")

# Check for outliers
Q1 = df['charges'].quantile(0.25)
Q3 = df['charges'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['charges'] < lower_bound) | (df['charges'] > upper_bound)]
print(f"Number of outliers in charges: {len(outliers)}")
print(f"Percentage of outliers: {(len(outliers)/len(df)*100):.2f}%")

In [ ]:
# Select only numeric columns for correlation
numeric_data = df.select_dtypes(include=['int64', 'float64'])

# Create correlation matrix
corr_matrix = numeric_data.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1)
plt.title('Correlation Heatmap - Numeric Features')
plt.tight_layout()
plt.show()

print("\n")

# Show correlation with charges in detail
print("Correlation with charges (sorted):")
print(corr_matrix['charges'].sort_values(ascending=False))

In [ ]:
# Separate features and target
X = df.drop('charges', axis=1)
y = df['charges']

print("Features (X):")
print(X.head())

print("\n")

print("Target (y):")
print(y.head())

print("\n")

# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"\nCategorical features ({len(categorical_features)}): {categorical_features}")

print("\n")

# Check categorical feature
print("Categorical features - unique values:")
for col in categorical_features:
    print(f"{col}: {X[col].nunique()} unique values")

In [ ]:
# Numeric transformer - impute + scale
num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

# Categorical transformer - impute + encode
cat_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]
)

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numeric_features),
        ('cat', cat_transformer, categorical_features)
    ]
)

print("Preprocessing pipeline created!")
print("\nNumeric features will be:")
print("  - Imputed with median")
print("  - Scaled with StandardScaler")

print("\nCategorical features will be:")
print("  - Imputed with most_frequent")
print("  - Encoded with OneHotEncoder")

print("\n")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Train-test ratio: 80-20")

In [ ]:
# Define base models
reg_lr = LinearRegression()
reg_rf = RandomForestRegressor(n_estimators=100, random_state=42)
reg_gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
reg_knn = KNeighborsRegressor(n_neighbors=5,weights='distance')

# Voting Regressor - average of all models
voting_reg = VotingRegressor(
    estimators=[
        ('lr', reg_lr),
        ('rf', reg_rf),
        ('gb', reg_gb),
        ('knn', reg_knn),
    ]
)

# Stacking Regressor - meta learner
stacking_reg = StackingRegressor(
    estimators=[
        ('rf', reg_rf),
        ('gb', reg_gb),
        ('knn', reg_knn),

    ],
    final_estimator=Ridge()
)

# Dictionary of all models
models_to_train = {
    'Linear Regression': reg_lr,
    'Random Forest': reg_rf,
    'Gradient Boosting': reg_gb,
    'KNN Regression': reg_knn,
    'Voting Ensemble': voting_reg,
    'Stacking Ensemble': stacking_reg
}

print("6 models defined!")
print("\nModels to train:")
for i, name in enumerate(models_to_train.keys(), 1):
    print(f"  {i}. {name}")

print("\n")

# Train and evaluate all models
results = []

for name, model in models_to_train.items():
    print(f"Training {name}...")

    # Create pipeline
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Train
    pipe.fit(X_train, y_train)

    # Predict
    y_pred = pipe.predict(X_test)

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    results.append({
        'Model': name,
        'R2 Score': r2,
        'RMSE': rmse,
        'MAE': mae
    })

    print(f"{name} trained - R2: {r2:.4f}")

print("\n")

# Create results dataframe
results_df = pd.DataFrame(results).sort_values('R2 Score', ascending=False)

print("MODEL COMPARISON RESULTS:")
print(results_df.to_string(index=False))

In [ ]:
# Select best model
best_model_name = results_df.iloc[0]['Model']
best_model_obj = models_to_train[best_model_name]

print(f"Best Model: {best_model_name}")
print(f"R2 Score: {results_df.iloc[0]['R2 Score']:.4f}")
print(f"RMSE: {results_df.iloc[0]['RMSE']:.2f}")
print(f"MAE: {results_df.iloc[0]['MAE']:.2f}")

print("\n")

# Train best model on full pipeline
final_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model_obj)
])

final_pipe.fit(X_train, y_train)
y_final_pred = final_pipe.predict(X_test)

# Plot Actual vs Predicted
plt.figure(figsize=(10, 6))

sns.scatterplot(x=y_test, y=y_final_pred, alpha=0.6, color='teal')
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         color='red', linestyle='--', linewidth=2, label='Perfect Prediction')

plt.xlabel('Actual charges', fontsize=12)
plt.ylabel('Predicted charges', fontsize=12)
plt.title(f'Actual vs Predicted charges - {best_model_name}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n")

# Show some sample predictions
sample_results = pd.DataFrame({
    'Actual charges': y_test.head(10).values,
    'Predicted charges': y_final_pred[:10].astype(int),
    'Difference': (y_test.head(10).values - y_final_pred[:10]).astype(int)
})

print("Sample Predictions (First 10):")
print(sample_results.to_string(index=False))

In [ ]:
# Cross-validation with Gradient Boosting
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(n_estimators=100,random_state=42))
])

print("Performing 5-Fold Cross-Validation on Gradient Boosting...")

# 5-fold cross-validation
cv_scores = cross_val_score(
    gb_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring='neg_mean_squared_error'
)

cv_rmse = np.sqrt(-cv_scores)

print("Cross-Validation Results:")
print(f"RMSE scores for each fold: {cv_rmse}")
print(f"\nMean RMSE: {cv_rmse.mean():.2f}")
print(f"Std RMSE: {cv_rmse.std():.2f}")

print("\n")

# Also check R2 with cross-validation
cv_r2_scores = cross_val_score(
    gb_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring='r2'
)

print("R2 scores for each fold:", cv_r2_scores)
print(f"Mean R2: {cv_r2_scores.mean():.4f}")
print(f"Std R2: {cv_r2_scores.std():.4f}")

In [ ]:
# Train
gb_pipeline.fit(X_train, y_train)

# Predictions
y_train_pred = gb_pipeline.predict(X_train)
y_test_pred  = gb_pipeline.predict(X_test)


# Metrics
def regression_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAE": mean_absolute_error(y_true, y_pred)
    }

train_metrics = regression_metrics(y_train, y_train_pred)
test_metrics  = regression_metrics(y_test, y_test_pred)


# Print Results
print("TRAIN PERFORMANCE")
print(f"R2   : {train_metrics['R2']:.4f}")
print(f"RMSE : {train_metrics['RMSE']:.2f}")
print(f"MAE  : {train_metrics['MAE']:.2f}")

print("\nTEST PERFORMANCE")
print(f"R2   : {test_metrics['R2']:.4f}")
print(f"RMSE : {test_metrics['RMSE']:.2f}")
print(f"MAE  : {test_metrics['MAE']:.2f}")


# Overfitting Check
print("\nOVERFITTING CHECK")
print(f"R2 Difference   : {train_metrics['R2'] - test_metrics['R2']:.4f}")
print(f"RMSE Difference: {test_metrics['RMSE'] - train_metrics['RMSE']:.2f}")
print(f"MAE Difference : {test_metrics['MAE'] - train_metrics['MAE']:.2f}")


In [ ]:
# Get before tuning metrics FIRST
print("Training baseline model...")
gb_pipeline.fit(X_train, y_train)
y_pred_before = gb_pipeline.predict(X_test)
before_r2 = r2_score(y_test, y_pred_before)
before_rmse = np.sqrt(mean_squared_error(y_test, y_pred_before))
before_mae = mean_absolute_error(y_test, y_pred_before)

print(f"Baseline Performance - R2: {before_r2:.4f}, RMSE: {before_rmse:.2f}, MAE: {before_mae:.2f}")
print("\n")

# Define parameter grid for RandomizedSearchCV
param_dist = {
    'model__n_estimators': randint(100, 300),
    'model__max_depth': [3, 5, 7, 10],
    'model__min_samples_split': randint(2, 10),
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2]
}

print("Starting Hyperparameter Tuning...")

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_pipeline,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Fit
random_search.fit(X_train, y_train)

print("\n")

print("Best Parameters Found:")
print(random_search.best_params_)

print(f"\nBest Cross-Validation RMSE: {-random_search.best_score_:.2f}")

print("\n")

# Evaluate tuned model on test set
y_tuned_pred = random_search.predict(X_test)

tuned_r2 = r2_score(y_test, y_tuned_pred)
tuned_rmse = np.sqrt(mean_squared_error(y_test, y_tuned_pred))
tuned_mae = mean_absolute_error(y_test, y_tuned_pred)

print("Tuned Model Performance on Test Set:")
print(f"R2 Score: {tuned_r2:.4f}")
print(f"RMSE: {tuned_rmse:.2f}")
print(f"MAE: {tuned_mae:.2f}")

print("\n")

# Compare before and after tuning
print("COMPARISON: Before vs After Tuning")
print(f"{'Metric':<15} {'Before':<15} {'After':<15} {'Improvement'}")
print("\n")
print(f"{'R2 Score':<15} {before_r2:<15.4f} {tuned_r2:<15.4f} {'+' if tuned_r2 > before_r2 else ''}{(tuned_r2 - before_r2):.4f}")
print(f"{'RMSE':<15} {before_rmse:<14.2f} {tuned_rmse:<14.2f} {before_rmse - tuned_rmse:.2f}")
print(f"{'MAE':<15} {before_mae:<14.2f} {tuned_mae:<14.2f} {before_mae - tuned_mae:.2f}")

In [ ]:
#best tuned model
best_model = random_search.best_estimator_

# Train predictions
y_train_pred = best_model.predict(X_train)

# Test predictions
y_test_pred = best_model.predict(X_test)


# Train metrics
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)

# Test metrics
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

print("TRAIN PERFORMANCE")
print(f"R2   : {train_r2:.4f}")
print(f"RMSE : {train_rmse:.2f}")
print(f"MAE  : {train_mae:.2f}")

print("\nTEST PERFORMANCE")
print(f"R2   : {test_r2:.4f}")
print(f"RMSE : {test_rmse:.2f}")
print(f"MAE  : {test_mae:.2f}")

print("\nOVERFITTING CHECK")
print(f"R2 Difference   : {train_r2 - test_r2:.4f}")
print(f"RMSE Difference: {test_rmse - train_rmse:.2f}")
print(f"MAE Difference : {test_mae - train_mae:.2f}")
